This script is about showing you how to setup postgres access using the crobat package
The advantages to postgres is that its a long term data storage engine
While I need to work on archiving old data, rdms is great for doing panalysis 

In [3]:
# lets try set up the environment.
import os, sys
os.chdir('../..')
#os.chdir('..') # moves us 2 folders back into to allow for imports that users would use
wd = os.getcwd()
print(wd)
sys.path.append(wd)
#print(os.getcwd())
for i in sys.path:
    print(i)

/home/ivan/Documents/github
/home/ivan/Documents/github/crobat/Jupyter nbs
/home/ivan/.vscode/extensions/ms-toolsai.jupyter-2021.6.999662501/pythonFiles/vscode_datascience_helpers
/home/ivan/.vscode/extensions/ms-toolsai.jupyter-2021.6.999662501/pythonFiles
/home/ivan/.vscode/extensions/ms-toolsai.jupyter-2021.6.999662501/pythonFiles/lib/python
/usr/lib/python38.zip
/usr/lib/python3.8
/usr/lib/python3.8/lib-dynload

/home/ivan/.local/lib/python3.8/site-packages
/usr/local/lib/python3.8/dist-packages
/usr/lib/python3/dist-packages
/home/ivan/.local/lib/python3.8/site-packages/IPython/extensions
/home/ivan/.ipython
/home/ivan/Documents/github


In [4]:
# lets try importing the postgres setup 
from crobat.postgres import recorder as rec 
from copra.websocket import Channel, Client

printing the sub sys path for the recorder module for postgres
/home/ivan/Documents/github/crobat/Jupyter nbs
/home/ivan/.vscode/extensions/ms-toolsai.jupyter-2021.6.999662501/pythonFiles/vscode_datascience_helpers
/home/ivan/.vscode/extensions/ms-toolsai.jupyter-2021.6.999662501/pythonFiles
/home/ivan/.vscode/extensions/ms-toolsai.jupyter-2021.6.999662501/pythonFiles/lib/python
/usr/lib/python38.zip
/usr/lib/python3.8
/usr/lib/python3.8/lib-dynload

/home/ivan/.local/lib/python3.8/site-packages
/usr/local/lib/python3.8/dist-packages
/usr/lib/python3/dist-packages
/home/ivan/.local/lib/python3.8/site-packages/IPython/extensions
/home/ivan/.ipython
/home/ivan/Documents/github
/home/ivan/Documents/github/crobat/crobat
/home/ivan/Documents/github/crobat/postgres
the last entry in sys.path is.... /home/ivan/Documents/github/tests


In [5]:
# Step 1 : defining settings class
class input_args(object):
    def __init__(self, currency_pair='ETH-USD', position_range=5, recording_duration=5):
        self.currency_pair = currency_pair
        self.position_range = position_range
        self.recording_duration = recording_duration

settings = input_args()
print(dir(settings))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'currency_pair', 'position_range', 'recording_duration']


In [6]:
# step 2 constructing the ws connection
import asyncio
import nest_asyncio
nest_asyncio.apply()
loop = asyncio.get_event_loop()
channel = Channel('level2', settings.currency_pair) 
#channel2 = Channel('ticker', settings.currency_pair)
ws = rec.Ticker(loop, channel, settings)
#ws.subscribe(channel2)
try:
    loop.run_forever()
except KeyboardInterrupt:
    loop.run_until_complete(ws.close())
    loop.close()

Connecting to the PostgreSQL database...
Let's count the L3 messages! 2021-08-16 17:17:26.328183
closed the cursor
Database connection closed.
Connection to server is closed
True
None
None


KeyboardInterrupt: 

In [12]:
# now we need to take a look at the database :)
import sqlconnection as sqlc
import pandas as pd
from datetime import timedelta
cur, conn = sqlc.open_SQL_connection()
read_instance = sqlc.psql_read_operations(cur, conn)

# custom sql fetch get a list of recorded change messages in the last 10 seconds
latest_stamp = read_instance.get_last_tstamp()
print(latest_stamp)
start_stamp = latest_stamp - timedelta(seconds=1)
print("1 seconds before the latest message message", start_stamp)
sql_cmd = """ SELECT changes AS TOTAL FROM messages WHERE tstamp BETWEEN '{}' AND '{}' ;""".format(start_stamp, latest_stamp)
out = read_instance.custom_sql_fetch(sql_cmd)
out_df = pd.DataFrame(out)
print(out_df)

Connecting to the PostgreSQL database...
2021-08-16 17:17:31.503428
2021-08-16 17:17:31.503428
1 seconds before the latest message message 2021-08-16 17:17:30.503428
multi item return of len:  99
       0        1            2
0   sell  3217.51   0.00000000
1    buy  3210.86   0.00000000
2   sell  3216.18   0.00000000
3    buy  3212.01   0.00000000
4   sell  3215.03   0.00000000
..   ...      ...          ...
94   buy  3212.30   1.44600000
95   buy  3212.93   0.00000000
96   buy  3211.74  12.60619982
97   buy  3213.08   0.47440000
98   buy  3213.07   0.83332028

[99 rows x 3 columns]
